## Preambule

In [1]:
# General packages
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
import xarray as xr
import json

# Plotting
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.colors import n_colors

## Read

In [2]:
path_main = Path("K:/Code/EffortSharing/")
path_data = Path("K:/Data/Data_EffortSharing/DataUpdate_ongoing/startyear_2021/")
path_data_gen = Path("K:/Data/Data_EffortSharing/DataUpdate_ongoing/")
path_figs = path_main / "Figures" / "NZreport"
name_gas = 'GHG'
name_lulucf = 'incl'
name_histvar = name_gas + "_hist"
if name_lulucf == 'excl':
    name_histvar = name_gas + "_hist_" + name_lulucf

In [3]:
xr_dataread = xr.open_dataset(path_data / "xr_dataread.nc")
all_regions_iso = np.load(path_data_gen / "all_regions.npy")
all_regions_names = np.load(path_data_gen / "all_regions_names.npy")
all_countries_iso = np.load(path_data_gen / "all_countries.npy", allow_pickle=True)
all_countries_names = np.load(path_data_gen / "all_countries_names.npy", allow_pickle=True)

In [4]:
settings_default =  {'Temperature': [1.6],
                   'Risk': [0.5],
                   'NegEmis': 0.3,
                   'NonCO2red': [0.33, 0.5, 0.67],
                   'Timing': "Immediate", 
                   'Convergence_year': [2040],
                   'Discount_factor': [0.0],
                   'Historical_startyear': 1990}

In [5]:
major_countries = ['USA', 'JPN', 'EU', 'RUS', 'ZAF', 'CHN', 'BRA', 'IND'][::-1]

In [6]:
xr_allocs = []
for cty in major_countries:#list(all_countries_iso)+['EU']+['EARTH']:
    xr_allocs.append(xr.open_dataset(path_data / ("Allocations_"+name_gas+"_"+name_lulucf) / ("xr_alloc_"+cty+".nc")).expand_dims({'Region':[cty]}).sel(**settings_default)[['GF', 'ECPC', 'PCC', 'AP', 'PC']])
xr_allocs = xr.merge(xr_allocs)

# xr_allocs_incl = []
# for cty in ['EU', 'USA', 'IND', 'CHN', 'JPN', 'AUS', 'BRA', 'RUS', 'ZAF']:
#     xr_allocs_incl.append(xr.open_dataset(path_data / "Allocations_CO2_incl" / ("xr_alloc_"+cty+".nc")).expand_dims({'Region':[cty]})[['GF', 'ECPC', 'PCC', 'AP', 'PC']].sel(Temperature=1.6, Risk=0.5, NegEmis=0.5, NonCO2red=0.5, Timing='Immediate', Scenario='SSP2'))
# xr_allocs_incl = xr.merge(xr_allocs_incl)

In [7]:
reds_2040 = -(xr_allocs.sel(Time=2040) - xr_dataread.sel(Time=2015)[name_histvar]) / xr_dataread.sel(Time=2015)[name_histvar]
reds_2030 = -(xr_allocs.sel(Time=2030) - xr_dataread.sel(Time=2015)[name_histvar]) / xr_dataread.sel(Time=2015)[name_histvar]

In [8]:
df_nzscen = pd.read_excel("X:/user/dekkerm/Data/ScenarioOutput/ELV-SSP2-NDC-LTS.xlsx", sheet_name="data")
df_nzscen = df_nzscen[['Region', str(2020), str(2025), str(2030), str(2035), str(2040), str(2045), str(2050)]]

# melt years into a single column time
df_nzscen = df_nzscen.melt(id_vars=['Region'], var_name='Time', value_name='Cumulative emissions')
df_nzscen['Time'] = df_nzscen['Time'].astype(int)
xr_nzscen = xr.Dataset.from_dataframe(df_nzscen.set_index(['Region', 'Time']))
xr_nzscen = xr_nzscen.reindex({'Time':np.arange(2020, 2051)})

# Interpolate
xr_nzscen = xr_nzscen.interpolate_na(dim='Time', method='linear')
xr_nzscen = xr_nzscen.sel(Time=2050) - xr_nzscen.sel(Time=2021)
xr_nzscen = xr_nzscen.reindex({'Region': np.unique(list(np.array(xr_nzscen.Region)) + list(all_regions_iso))})

xr_nzscen.loc[{'Region': 'EU'}] = xr_nzscen.sel(Region='CEU') +  xr_nzscen.sel(Region='WEU')
xr_nzscen.loc[{'Region': 'ZAF'}] = xr_nzscen.sel(Region='SAF')
xr_nzscen.loc[{'Region': 'JPN'}] = xr_nzscen.sel(Region='JAP')
xr_nzscen.loc[{'Region': 'AUS'}] = xr_nzscen.sel(Region='OCE')
xr_nzscen.loc[{'Region': 'IND'}] = xr_nzscen.sel(Region='INDIA')

In [9]:
xr_allocs2 = []
for cty in major_countries:#list(all_countries_iso)+['EU']+['EARTH']:
    xr_allocs2.append(xr.open_dataset(path_data / ("Allocations_CO2_incl") / ("xr_alloc_"+cty+".nc")).expand_dims({'Region':[cty]}).sel(**settings_default)[['GF', 'ECPC', 'PCC', 'AP', 'PC']])
xr_allocs2 = xr.merge(xr_allocs2)

# df_nz = pd.read_csv("X:/user/dekkerm/Data/NetZeroYears/nztargetyears.csv")
# xr_nz = xr.Dataset.from_dataframe(df_nz.set_index('iso3c')[['end_target_year_comb']]).rename({'iso3c': "Region", "end_target_year_comb": "NZ_year"})
# regs = np.array(xr_nz.Region)
# regs[regs == 'FRA'] = 'EU'
# xr_nz = xr_nz.assign_coords({'Region': regs})
policy_budget = xr_nzscen['Cumulative emissions']#(xr_nz.NZ_year - 2021)*0.5*xr_dataread[name_histvar].sel(Time=2021)

xr_bud = xr_allocs2.sel(Time=np.arange(2021, 2051)).sum(dim='Time')
#xr_bud3 = xr_allocs2.where(xr_allocs2.Time <= xr_nz.NZ_year).sum(dim='Time')
debtpc = (xr_bud - policy_budget) / xr_dataread.Population.sel(Time=2021, Scenario='SSP2')

In [10]:
vals_2030_pc = []
vals_2030_ecpc = []
vals_2030_ap = []
vals_2040_pc = []
vals_2040_ecpc = []
vals_2040_ap = [] 
regs_pcc = []
regs_ecpc = []
regs_ap = []
debt_pc = []
debt_ecpc = []
debt_ap = []
for cty in major_countries:
    vals_2030_pc = vals_2030_pc + list(reds_2030.sel(Region=cty).PCC.values.flatten())
    vals_2030_ecpc = vals_2030_ecpc + list(reds_2030.sel(Region=cty).ECPC.values.flatten())
    vals_2030_ap = vals_2030_ap + list(reds_2030.sel(Region=cty).AP.values.flatten())

    vals_2040_pc = vals_2040_pc + list(reds_2040.sel(Region=cty).PCC.values.flatten())
    vals_2040_ecpc = vals_2040_ecpc + list(reds_2040.sel(Region=cty).ECPC.values.flatten())
    vals_2040_ap = vals_2040_ap + list(reds_2040.sel(Region=cty).AP.values.flatten())

    debt_pc = debt_pc + list(debtpc.sel(Region=cty).PC.values.flatten())
    debt_ecpc = debt_ecpc + list(debtpc.sel(Region=cty).ECPC.values.flatten())
    debt_ap = debt_ap + list(debtpc.sel(Region=cty).AP.values.flatten())

    regs_pcc = regs_pcc + [all_regions_names[all_regions_iso == cty][0]]*len(reds_2030.sel(Region=cty).PCC.values.flatten())
    regs_ecpc = regs_ecpc + [all_regions_names[all_regions_iso == cty][0]]*len(reds_2030.sel(Region=cty).ECPC.values.flatten())
    regs_ap = regs_ap + [all_regions_names[all_regions_iso == cty][0]]*len(reds_2030.sel(Region=cty).AP.values.flatten())

    #vals_2040 = vals_2040 + list(reds_2030.PCC.values.flatten())+list(reds_2030.ECPC.values.flatten())+list(reds_2030.AP.values.flatten())
    #regs = regs + [cty]*(len(reds_2030.PCC.values.flatten())+len(reds_2030.ECPC.values.flatten())+len(reds_2030.AP.values.flatten()))

## Plot

In [12]:
fig = make_subplots(rows=1, cols=3,
                    specs=[[{}, {}, {}]],
                    subplot_titles=("<b>GHG reduction targets<br>by 2030", "<b>GHG reduction targets<br>by 2040", "<b>Cumulative CO<sub>2</sub><br>emissions up to 2050"),
                    #subplot_titles=("<b>GHG reduction targets<br>by 2030", "<b>GHG reduction targets<br>by 2040", "<b>Cumulative GHG</sub><br>emissions"),
                    horizontal_spacing = 0.05, vertical_spacing=0.05, shared_xaxes=False, shared_yaxes=True)

col1 = "#afc50f"#"rgba(255, 65, 54, 1)"
col2 = '#2e5061'#'rgba(61, 153, 112, 1)'
col3 = '#0062a6'

fig.add_trace(go.Box(x=vals_2030_pc, y=regs_pcc, name = 'Equality', boxpoints=False, offsetgroup='A', marker_color=col3, line_width=3), 1, 1)
fig.add_trace(go.Box(x=vals_2030_ap, y=regs_ap,name = 'Capability',boxpoints=False, offsetgroup='B', marker_color=col1, line_width=3), 1, 1)
fig.add_trace(go.Box(x=vals_2030_ecpc, y=regs_ecpc, name = 'Responsibility',boxpoints=False, offsetgroup='C', marker_color=col2, line_width=3), 1, 1) #rgba(255, 133, 27,1)

fig.add_trace(go.Box(x=vals_2040_pc, y=regs_pcc, offsetgroup='A', showlegend=False, boxpoints=False, marker_color=col3, line_width=3), 1, 2)
fig.add_trace(go.Box(x=vals_2040_ap, y=regs_ap, offsetgroup='B', showlegend=False, boxpoints=False, marker_color=col1, line_width=3), 1, 2)
fig.add_trace(go.Box(x=vals_2040_ecpc, y=regs_ecpc, offsetgroup='C', showlegend=False, boxpoints=False,  marker_color=col2, line_width=3), 1, 2)

fig.add_vline(x=0.25, line_width=1, line_dash="solid", line_color="black")
fig.add_vline(x=0.5, line_width=1, line_dash="solid", line_color="black")
fig.add_vline(x=0.75, line_width=1, line_dash="solid", line_color="black")
fig.add_vline(x=1.25, line_width=1, line_dash="solid", line_color="black")
fig.add_vline(x=1, line_width=4, line_dash="dash", line_color="black")

fig.add_trace(go.Box(x=debt_pc, y=regs_pcc, offsetgroup='A', showlegend=False, boxpoints=False, marker_color=col3, line_width=3), 1, 3)
fig.add_trace(go.Box(x=debt_ap, y=regs_ap, offsetgroup='B', showlegend=False, boxpoints=False, marker_color=col1, line_width=3), 1, 3)
fig.add_trace(go.Box(x=debt_ecpc, y=regs_ecpc, offsetgroup='C', showlegend=False, boxpoints=False,  marker_color=col2, line_width=3), 1, 3)
fig.add_vline(x=0, line_width=4, line_dash="solid", line_color="black")

fig.update_layout(
    boxmode='group', # group together boxes of the different traces for each value of x
      boxgap=0.25,       # Space between different categories (days)
    boxgroupgap=0.1,
        yaxis=dict(tickfont=dict(size=18)),   # Increase tick font size for first subplot
        yaxis2=dict(tickfont=dict(size=18), showline=False, ticks=''),
        yaxis3=dict(tickfont=dict(size=18), showline=False, ticks=''),
        xaxis=dict(tickfont=dict(size=18)),   # Increase tick font size for first subplot
        xaxis2=dict(tickfont=dict(size=18)),
        xaxis3=dict(tickfont=dict(size=18))
)

for i in range(3):
  fig.update_xaxes(
      tickvals=[[0, 0.25, 0.5, 0.75, 1.0, 1.25], [0, 0.25, 0.5, 0.75, 1.0, 1.25], [-300, -150, 0, 150]][i],  # Custom tick positions
      ticktext=[['0%', '25%', '50%', '75%', 'Net<br>zero', '125%'], ['0%', '25%', '50%', '75%', 'Net<br>zero', '125%'], ['300<br>t CO<sub>2</sub>/cap<br>debt', '150<br>t CO<sub>2</sub>/cap<br>debt', 'No debt<br>nor credit', '150<br>t CO<sub>2</sub>/cap<br>credit']][i],  # Custom tick labels
      row=1, col=1+i,
      range=[[-0.2, 1.5], [-0.2, 1.5], [-300, 150]][i],
      title_text = ["Reduction relative to 2015", "Reduction relative to 2015", "Carbon debt (left) or credit (right) per capita"][i],
  )

fig.update_traces(orientation='h')
for annotation in fig['layout']['annotations']:
    annotation['font'] = dict(size=25)
fig.update_layout(height=900, width=1600, template='simple_white',
                  legend=dict(
                     x=-0.15, y=1.12,
        font=dict(size=20)
    ))
fig.write_image(path_figs / "Figure_2_h2.svg", scale=4)
fig.show()